In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings 
warnings.filterwarnings("ignore")
import seaborn as sns
os.chdir('C:\\Users\\avinash\\Desktop')
data=pd.read_csv('bigmart_data.csv')
data.drop(columns='Item_Identifier',inplace=True)

In [6]:
y=data['Item_Outlet_Sales']
data.drop(columns='Item_Outlet_Sales',inplace=True)

In [7]:
na_df=pd.DataFrame({"Column_name":data.columns,
                     "Na_Count":data.isnull().sum(),
                     "Na_Percentage":data.isnull().sum()/data.shape[0]*100})
na_df
na_df.sort_values(by="Na_Percentage",ascending=False)

,Column_name,Na_Count,Na_Percentage
Outlet_Size,Outlet_Size,2410,28.276428
Item_Weight,Item_Weight,1463,17.165317
Item_Fat_Content,Item_Fat_Content,0,0.000000
Item_Visibility,Item_Visibility,0,0.000000
Item_Type,Item_Type,0,0.000000
Item_MRP,Item_MRP,0,0.000000
Outlet_Identifier,Outlet_Identifier,0,0.000000
Outlet_Establishment_Year,Outlet_Establishment_Year,0,0.000000
Outlet_Location_Type,Outlet_Location_Type,0,0.000000
Outlet_Type,Outlet_Type,0,0.000000


In [8]:
print(data['Outlet_Size'].dtype)
print(data['Item_Weight'].dtype)
print(data['Item_Weight'].mean())
print(data['Item_Weight'].median())

object
float64
12.857645184136183
12.6


In [10]:
data['Outlet_Size'].fillna(data['Outlet_Size'].mode()[0],inplace=True)
data['Item_Weight'].fillna(data['Item_Weight'].median(),inplace=True)

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data,y,train_size=0.3,random_state=42)



In [20]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
label=LabelEncoder()
catgeroical_cols=[col for col in X_train.columns if X_train[col].dtype=="object"]
catgeroical_cols

for ele in  catgeroical_cols:
    X_train[ele]=label.fit_transform(X_train[ele])
    X_test[ele]=label.transform(X_test[ele])

In [21]:
from sklearn.preprocessing import StandardScaler
import numpy as np
scaler=StandardScaler()
continous_cols=[col for col in X_train.columns if X_train[col].dtype=="float64" or X_train[col].dtype=="int64"]
continous_cols

for ele in continous_cols:
    X_train[ele]=scaler.fit_transform(np.array(X_train[ele]).reshape(-1,1))
    X_test[ele]=scaler.transform(np.array(X_test[ele]).reshape(-1,1))

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error,adjusted_rand_score
reg=LinearRegression()
reg.fit(X_train,y_train)
y_pred=reg.predict(X_test)

print(mean_squared_error(y_test,y_pred))
print(r2_score(y_test,y_pred))

1426906.7433568435
0.505999875974591


In [23]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
tree=DecisionTreeRegressor(random_state=42)
tree
parameter={"criterion":["mse"],
            "min_samples_split":[2,3,4,5,6],#here min samples split should be greater than 1 otherwise it will raise a error
            "min_samples_leaf":[5,6,7,8,9],
            "max_depth":[5,6,7,8,9]}
rd_cv=RandomizedSearchCV(estimator=tree,param_distributions=parameter,cv=5,n_iter=80)
rd_cv.fit(X_train,y_train)

print(rd_cv.best_params_)

print(rd_cv.best_score_)

{'min_samples_split': 4, 'min_samples_leaf': 9, 'max_depth': 5, 'criterion': 'mse'}
0.5733157152396365


In [24]:
gd_cv=GridSearchCV(estimator=tree,param_grid=parameter,cv=5)
gd_cv.fit(X_train,y_train)
print(gd_cv.best_params_)
print(gd_cv.best_score_)

{'criterion': 'mse', 'max_depth': 5, 'min_samples_leaf': 9, 'min_samples_split': 2}
0.5733157152396365


In [25]:
tree=DecisionTreeRegressor(criterion='mse',min_samples_split=6,min_samples_leaf=9,random_state=52,max_depth=5)
tree
tree.fit(X_train,y_train)
y_pred1=tree.predict(X_test)
print(mean_squared_error(y_test,y_pred1))
print(r2_score(y_test,y_pred1))

1186220.1549015203
0.5893264178819255


In [26]:
from sklearn.ensemble import AdaBoostRegressor
ada=AdaBoostRegressor(base_estimator=tree,n_estimators=50,random_state=40)
ada
ada.fit(X_train,y_train)
y_pred2=ada.predict(X_test)
print(mean_squared_error(y_test,y_pred2))
print(r2_score(y_test,y_pred2))

1522687.8983813282
0.4728401038439469
